In [34]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy import linalg
import matplotlib.pyplot as plt
import random
from IPython.display import display, clear_output
from ipywidgets import widgets, Layout, interact, interactive, fixed, interact_manual
from jupyterthemes import jtplot
import time
import scipy as scipy
import csv
# import cufflinks as cf 
# import plotly.offline as py

# cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)


In [47]:
# Input the initial variables:
style = {'description_width' : 'initial'}
erosion_type = widgets.Dropdown(options = [('Stochastic Rockfalls', 'stochastic'),
                                           ('Steady-state', 'constant'),
                                           ('No Erosion', 'none')],
                                value='stochastic',
                                description='Erosion type:',
                                style = style)

print('Select an erosion type for the simulation.')
display(erosion_type)

total_time = widgets.BoundedIntText(value = 100,
                                   min = 10,
                                   max = 150000,
                                   description = "Total time:")

print('Input total time length (years) for individual simulations. \nMax = 150000 \nMin = 10')
display(total_time)

SLHL_C14 = widgets.Dropdown(options=[('14.1 \t (Author et al. (2010)', 14.1),
                                      ('13.2 \t (Author et al. (1992)', 13.2),
                                      ('12.1 \t (Author et al. (1999)', 12.1)],
                           value=14.1,
                           description='$^{14}$C Production Rate:',
                           style = style)
print('Select a C-14 sea level high latitude production rate. \nAuthor et al. (2016) (14.1) is recommended.')
display(SLHL_C14)

SLHL_Be10 = widgets.Dropdown(options=[('4.1 \t (Author et al. (2010)', 4.1),
                                      ('3.95 \t (Author et al. (1992)', 3.95),
                                      ('3.8 \t (Author et al. (1999)', 3.8)],
                           value=4.1,
                           description='$^{10}$Be Production Rate:',
                           style = style)
print('Select a Be-10 sea level high latitude production rate. \nAuthor et al. (2016) (4.1) is recommended.')
display(SLHL_Be10)

SLHL_He3 = widgets.Dropdown(options=[('114.2 \t (Author et al. (2010)', 114.2),
                                      ('120.5 \t (Author et al. (1992)', 120.5),
                                      ('132.4 \t (Author et al. (1999)', 132.4)],
                           value=120.5,
                           description='$^{3}$He Production Rate:',
                           style = style)
print('Select a He-3 sea level high latitude production rate. \nAuthor et al. (2010) (120.5) is recommended.')
display(SLHL_He3)

scaling_factor = widgets.BoundedFloatText(value = 1.1,
                                min = 0.0000001,
                                max = 900.0,
                                step = 0.1, 
                                description = "CN scaling factor: \t",
                                style = style)
print('Input cosmogenic nuclide production rate scaling factor. \nFor SLHL, enter ' + str('1') + '.')
display(scaling_factor)

gr_sz = widgets.BoundedFloatText(value = 0.1,
                                min = 0.08,
                                max = 2.0,
                                step = 0.01, 
                                description = "Quartz grain size",
                                style = style)
print('Input the mean grain size of the quartz used for He-3 measurements.')
display(gr_sz)

MAT = widgets.BoundedFloatText(value = 0.0,
                                min = -50.0,
                                max = 15.0,
                                step = 0.5, 
                                description = "Mean annual temperature (0 $\circ$C)",
                                style = style)
print('Input a mean annual temperature for the simulations.')
display(MAT)

T_AMP=widgets.BoundedFloatText(value=5.0,
                                min=0.5,
                                max=35.0,
                                step=0.5, 
                                description = "Temperature seasonality",
                                style = style)
print('Input an amplitude for the seasonal temperature function. \nAmplitude of 5.0 for MAT = 0.0 indicates mean summer temperature of +5.0 C and mean winter temperature of -5.0 C.')
display(T_AMP)

save_output = widgets.Dropdown(options = [('Yes', True), ('No',False)],
                              value = False,
                              description = 'Save Output')
print('Would you like to save the output files?')
display(save_output)

save_only_surf = True

Select an erosion type for the simulation.


Dropdown(description='Erosion type:', options=(('Stochastic Rockfalls', 'stochastic'), ('Steady-state', 'const…

Input total time length (years) for individual simulations. 
Max = 150000 
Min = 10


BoundedIntText(value=100, description='Total time:', max=150000, min=10)

Select a C-14 sea level high latitude production rate. 
Author et al. (2016) (14.1) is recommended.


...

BoundedFloatText(value=0.0, description='Mean annual temperature (0 $\\circ$C)', max=15.0, min=-50.0, step=0.5…

Input an amplitude for the seasonal temperature function. 
Amplitude of 5.0 for MAT = 0.0 indicates mean summer temperature of +5.0 C and mean winter temperature of -5.0 C.


BoundedFloatText(value=5.0, description='Temperature seasonality', max=35.0, min=0.5, step=0.5, style=Descript…

Would you like to save the output files?


Dropdown(description='Save Output', index=1, options=(('Yes', True), ('No', False)), value=False)

In [48]:
# Input the case-dependent variables:
if erosion_type.value == 'stochastic':
    scenarios = widgets.BoundedIntText(value = 2,
                                   min = 1,
                                   max = 30,
                                   description = 'Scenarios:',
                                   style = style)
    print('Input number of stochastic erosion scenarios. \nMax = 30 \nMin = 1')
    display(scenarios)
    
    stoch_base_ER = widgets.BoundedFloatText(value = 0.1,
                                   min = 0.00001,
                                   max = 349.0,
                                   description = 'Base Erosion Rate (cm yr$^{-1}$:',
                                   style = style)
    print('Input the time-averaged erosion rate (total lost / total time).')
    display(stoch_base_ER)
    
    sample_no = widgets.BoundedIntText(value = 30,
                                      min = 2,
                                      max = (total_time.value - 1),
                                      description = 'No. of Sub-samples:',
                                      style = style)
    print('Input the number of sub-samples you would like to base the CN measurements on for each stochastic sample. These concentrations are selected from random times in the model run, as in a "time for space" substition.') 
    display(sample_no)
    

if erosion_type.value == 'constant':
    erosion_rate = widgets.FloatText(value = 0.1,
                                     min = 0.001,
                                     max = 350,
                                     description = 'Erosion Rate (cm yr$^{-1}$)',
                                    style = style)
    print('Input the steady-state erosion rate.')
    display(erosion_rate)

    shift_ER = widgets.Dropdown(options = [('Yes', True), ('No', False)],
                               value = False,
                               description = 'Change erosion rate mid-simulation?',
                               style = style)

    print('Would you like to change the erosion rate during the simulation? This will be a step-function change at a specific time.')  
    display(shift_ER)

    time_ER_shift = widgets.BoundedIntText(value = int(0.5 * total_time.value),
                                          min = 0,
                                          max = (total_time.value - 1),
                                          description = 'Time for rate change:',
                                          style = style)
    new_ER = widgets.BoundedFloatText(value = 0.1,
                                      min = 0.0000001,
                                      max = 349,
                                      description = 'New erosion rate (cm yr$^{-1}$)',
                                     style = style)
    print('If you will change the erosion rate, please input the year of the time change and the new erosion rate.')
    display(time_ER_shift)
    display(new_ER)

Input number of stochastic erosion scenarios. 
Max = 30 
Min = 1


BoundedIntText(value=2, description='Scenarios:', max=30, min=1, style=DescriptionStyle(description_width='ini…

Input the time-averaged erosion rate (total lost / total time).


BoundedFloatText(value=0.1, description='Base Erosion Rate (cm yr$^{-1}$:', max=349.0, min=1e-05, style=Descri…

Input the number of sub-samples you would like to base the CN measurements on for each stochastic sample. These concentrations are selected from random times in the model run, as in a "time for space" substition.


BoundedIntText(value=30, description='No. of Sub-samples:', max=24999, min=2, style=DescriptionStyle(descripti…

In [49]:
def get_values(widget):
    out_val = widget.value
    return out_val

def adv_values():
    # more values setting
    max_depth = [350, 1500]
    rock_rho = 2.7
    CN_lambda = 160
    mu = rock_rho / CN_lambda
    dt = 1

    # He diffusion inputs
    rho = 2.65 # quartz density
    nx = 100
    r = gr_sz.value
    dx = (r / nx)
    x = np.arange(dx/2, r, dx)
    x_up = x + (dx/2)
    x_low = x - (dx/2)
    x_vol = [(4/3 * np.pi * x_up[i]**3) - (4/3 * np.pi * x_low[i]**3) for i in range(len(x))]
    shell_mass = [(x_vol[i] * rho) for i in range(len(x_vol))]

    # Bedrock thermal properties
    alpha = 1.5e-6
    
    P0_Be10 = SLHL_Be10.value * scaling_factor.value
    P0_He3 = SLHL_He3.value * scaling_factor.value
    
    adv_list = [max_depth, rock_rho, CN_lambda, mu, dt, rho, nx, r, dx, x, x_up, x_low, x_vol, shell_mass, alpha, P0_Be10, P0_He3] 
    
    return adv_list

def set_values():
    var_list = [erosion_type, total_time, SLHL_C14, SLHL_Be10, SLHL_He3,
                scaling_factor, gr_sz, MAT, T_AMP, save_output]
    set_list = [get_values(var_list[i]) for i in range(len(var_list))]
    
    more_values = adv_values()
    
    if set_list[0] == 'stochastic':
        set_list.append(get_values(scenarios))
        set_list.append(get_values(stoch_base_ER))
        set_list.append(get_values(sample_no))
        set_list.extend(more_values)
    
    if set_list[0] =='constant':
        set_list.append(get_values(erosion_rate))
        set_list.append(get_values(shift_ER))
        set_list.append(get_values(time_ER_shift))
        set_list.append(get_values(new_ER) / get_values(erosion_rate))
        set_list.extend(more_values)
        
    return set_list

def post_process(CRN_output, He3_output):
    
    He3_percent_mat = np.empty((total_time, scenarios))
    
    for i in range(scenarios):
        He3_percent_mat[:,i] = np.divide(He3_output[1][:,i], CRN_output[1][:,1,i]) * P0_Be10 / P0_He3
    
    master_array = np.empty((total_time, 11, scenarios))
    
    for i in range(scenarios):
        master_array[:,:3,i] = CRN_output[1][:,:,i]
        master_array[:,3,i] = He3_output[1][:,i]
        master_array[:,4,i] = He3_percent_mat[:,i]
        master_array[:,5,i] = rockfall_output[:,i]
        master_array[:,6:,i] = comparison_constants[:,:,i]

    master_df = [pd.DataFrame(data = master_array[:,:,i],
                index = np.arange(0,total_time,1),
                columns = ['C-14 Conc. Stoch', 
                           'Be-10 Conc. Stoch', 
                           'C-14/Be-10 Ratio Stoch',
                           'He-3 Conc. Stoch',
                           'Perc. He-3 Stoch',
                           'Rockfall Size',
                           'C-14 Conc. Constant',
                           'Be-10 Conc. Constant',
                           'C-14/Be-10 Ratio Constant',
                           'He-3Conc. Constant',
                           'Perc. He-3 Constant']) for i in range(scenarios)]

    
    return master_array, master_df


In [ ]:
if get_values(erosion_type) == 'stochastic':
    [erosion_type, 
     total_time, 
     SLHL_C14, 
     SLHL_Be10,
     SLHL_He3, 
     scaling_factor, 
     gr_sz, MAT, T_AMP, 
     save_output,
     scenarios,
     stoch_base_ER, sample_no,
     max_depth, 
     rock_rho, 
     CN_lambda, 
     mu, dt, rho, 
     nx, r, dx, x, 
     x_up, x_low, 
     x_vol, shell_mass, 
     alpha, P0_Be10, P0_He3] = [set_values()[i] for i in range(len(set_values()))]
    
    # %run D:\STEIN_paper\model_outputs\scripts\rockfall_generation_stochastic_erosion.ipynb
    
    # rockfall_matrix_gen(stoch_base_ER)
    # rockfall_output = erosion_export(stoch_base_ER)
    
   #  %run D:\STEIN_paper\model_outputs\scripts\C14_Be10_stochastic_erosion.ipynb
    
    # rockfalls_imported = rockfalls_import()
    # CRN_output = CRN_loop_fxn(total_time)
    
    # %run D:\STEIN_paper\model_outputs\scripts\He3_stochastic_erosion.ipynb
    
    # He3_output = He3_loop_fxn(total_time)
    
    %run D:\STEIN_paper\model_outputs\scripts\stochastic_measured_constants_CRN_loops.ipynb
    
    comparison_constants = compare_fxn()
    
    master_array, master_df = post_process(CRN_output, He3_output)
    
elif get_values(erosion_type) == 'constant':
     
    [erosion_type, 
     total_time, 
     SLHL_C14, 
     SLHL_Be10,
     SLHL_He3, 
     scaling_factor, 
     gr_sz, MAT, T_AMP, 
     save_output, ER, 
     shift_ER, time_ER_shift,
     ER_shift_factor,
     max_depth, 
     rock_rho, 
     CN_lambda, 
     mu, dt, rho, 
     nx, r, dx, x, 
     x_up, x_low, 
     x_vol, shell_mass, 
     alpha, P0_Be10, P0_He3] = [set_values()[i] for i in range(len(set_values()))]    
    
    %run D:\STEIN_paper\model_outputs\scripts\C14_Be10_constant_erosion.ipynb
    %run D:\STEIN_paper\model_outputs\scripts\He3_constant_erosion.ipynb

C:\Users\dennis\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning:

invalid value encountered in true_divide



Comparison Be10 / C14 loop scenario 24999complete.
Comparison He3 loop scenario 24999complete.


C:\Users\dennis\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning:

invalid value encountered in true_divide



Comparison Be10 / C14 loop scenario 24999complete.
Comparison He3 loop scenario 24999complete.


C:\Users\dennis\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning:

invalid value encountered in true_divide



Comparison Be10 / C14 loop scenario 24999complete.
Comparison He3 loop scenario 24999complete.


C:\Users\dennis\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning:

invalid value encountered in true_divide



Comparison Be10 / C14 loop scenario 24999complete.
Comparison He3 loop scenario 24999complete.


In [ ]:
[plt.plot(x = np.arange(0,total_time,1), y = master_array[:,0,i]) for i in range(scenarios)]

In [ ]:
[ax1.plot(x = np.arange(0,total_time,1), y = master_array[:,0,i]) for i in range(scenarios)]

In [ ]:


















@interact
def scatter_plot(y1 = list(master_df[0].columns),
                 y2 = list(master_df[0].columns)):

    master_df[0].iplot(x = [np.arange(0,100,1)], y = [y1,y2])

In [ ]:
# master_array = np.empty((total_time, 11, scenarios))

# for i in range(scenarios):
#     master_array[:,:3,i] = CRN_output[1][:,:,i]
#     master_array[:,3,i] = He3_output[1][:,i]
#     master_array[:,4,i] = He3_percents(CRN_output, He3_output)[:,i]
#     # master_df[:,5,i] = rockfall_output[:,i]
#     master_array[:,6:,i] = comparison_constants[:,:,i]

# master_df = [pd.DataFrame(data = master_array[:,:,i],
#             index = np.arange(0,total_time,1),
#             columns = ['C-14 Conc. Stoch', 
#                        'Be-10 Conc. Stoch', 
#                        'C-14/Be-10 Ratio Stoch',
#                        'He-3 Conc. Stoch',
#                        'Perc. He-3 Stoch',
#                        'Rockfall Size',
#                        'C-14 Conc. Constant',
#                        'Be-10 Conc. Constant',
#                        'C-14/Be-10 Ratio Constant',
#                        'He-3Conc. Constant',
#                        'Perc. He-3 Constant']) for i in range(scenarios)]